In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
input_shape = (28, 28, 1)

x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0

In [ ]:
idx = np.argsort(y_train)
x_train = x_train[idx]
y_train = y_train[idx]

In [ ]:
idx1= np.argsort(y_test)
x_test = x_test[idx1]
y_test = y_test[idx1]

In [ ]:
print(y_test.shape)

(10000,)


In [ ]:
count=0
list1=[]
for j in range(10):
  list1.append(count)
  for i in y_train:
    if i == j:
      count = count+1
  print(count)

5923
12665
18623
24754
30596
36017
41935
48200
54051
60000


In [ ]:
count=0
list2=[]
for j in range(10):
  list2.append(count)
  for i in y_test:
    if i == j:
      count = count+1
  print(count)

980
2115
3147
4157
5139
6031
6989
8017
8991
10000


In [ ]:
list1.append(60000)
list2.append(10000)

In [ ]:
list1

[0, 5923, 12665, 18623, 24754, 30596, 36017, 41935, 48200, 54051, 60000]

In [ ]:
# y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

In [ ]:
iid_data_x = []
iid_data_y = []
for i in range(10):
  iid_data_x.append(x_train[list1[i]:list1[i+1]])
  iid_data_y.append(y_train[list1[i]:list1[i+1]])

In [ ]:
#iid_data_x.append(iid_data_x[0])

In [ ]:
import random

mixin_ratio = 0.9
clients = []
y_train = []
for i in range(10):
  empty = np.zeros((6000,28,28,1))
  num_samp = int(6000*mixin_ratio)
  samp_1 = i
  samp_2 = (i+1)%10
  y_temp= np.zeros((6000))
  y_temp[:num_samp] = samp_1
  y_temp[num_samp:] = samp_2
  y_temp = list(y_temp)
  
  y_train = y_train + y_temp
  
  empty[:num_samp] = iid_data_x[samp_1][:(num_samp)]
  empty[num_samp:] = iid_data_x[samp_2][:(6000-num_samp)]

  clients.append(empty)

In [ ]:
# import random

# mixin_ratio = 0.5
# clients = []
# y_train = []
# for i in range(10):
#   empty = np.zeros((6000,28,28,1))
#   num_samp = int(6000*mixin_ratio)
#   samp_1 = random.randint(0, 9)
#   samp_2 = random.randint(0, 9)
#   y_temp= np.zeros((6000))
#   y_temp[:num_samp] = samp_1
#   y_temp[:(6000-num_samp)] = samp_2
#   y_temp = list(y_temp)
  
#   y_train = y_train + y_temp
  
#   empty[:num_samp] = iid_data_x[samp_1][:(num_samp)]
#   empty[:(6000-num_samp)] = iid_data_x[samp_2][:(6000-num_samp)]

#   clients.append(empty)

In [ ]:
np.unique(np.array(y_train))

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [ ]:
y_train = tf.one_hot(np.array(y_train).astype(np.int32), depth=10)

In [ ]:
latent_dim = 64 

class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(64, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(784, activation='sigmoid'),
      layers.Reshape((28,28,1))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded
  
autoencoder = Autoencoder(latent_dim) 

In [ ]:
x_train_list = []
history_list = []
autoencoder_list = []
encoded_images_train = []
decoded_images_train = []
encoded_images_test = []
decoded_images_test = []

for i in range(10):

  autoencoder_list.append(Autoencoder(latent_dim)) 
  autoencoder_list[i].compile(optimizer=tf.keras.optimizers.Adam(), loss=losses.MeanSquaredError(),metrics=['acc'])

  autoencoder_list[i].fit(clients[i],clients[i],
                epochs=10)
  
  encoded_images_train.append(autoencoder_list[i].encoder(clients[i]).numpy())

  n = 10
  plt.figure(figsize=(20, 4))
  for j in range(n):
    # display original
    ax = plt.subplot(2, n, j + 1)
    plt.imshow(clients[i][j,:,:,0])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, j + 1 + n)
    plt.imshow(encoded_images_train[i].reshape(encoded_images_train[i].shape[0],8,8,1)[j,:,:,0])
    plt.title("reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
  plt.show()

Epoch 1/10
188/188 [==============================] - 4s 4ms/step - loss: 0.0592 - acc: 0.7412
Epoch 2/10
188/188 [==============================] - 1s 5ms/step - loss: 0.0281 - acc: 0.7675
Epoch 3/10
188/188 [==============================] - 1s 5ms/step - loss: 0.0201 - acc: 0.7717
Epoch 4/10
188/188 [==============================] - 1s 4ms/step - loss: 0.0158 - acc: 0.7735
Epoch 5/10
188/188 [==============================] - 1s 3ms/step - loss: 0.0132 - acc: 0.7746
Epoch 6/10
188/188 [==============================] - 1s 4ms/step - loss: 0.0115 - acc: 0.7753
Epoch 7/10
188/188 [==============================] - 1s 5ms/step - loss: 0.0101 - acc: 0.7757
Epoch 8/10
188/188 [==============================] - 1s 5ms/step - loss: 0.0091 - acc: 0.7760
Epoch 9/10
188/188 [==============================] - 1s 4ms/step - loss: 0.0083 - acc: 0.7762
Epoch 10/10
188/188 [==============================] - 1s 3ms/step - loss: 0.0076 - acc: 0.7764
Epoch 1/10
188/188 [=============================

KeyboardInterrupt: ignored

In [ ]:
overall_encoded_train = np.zeros((60000,latent_dim))

for i in range(10):
  overall_encoded_train[i*6000:(i+1)*6000] = encoded_images_train[i]

overall_encoded_train = overall_encoded_train.reshape((overall_encoded_train.shape[0], 8, 8, 1))

In [ ]:
# overall_encoded_train = np.zeros((60000,latent_dim))

# for i in range(10):
#   overall_encoded_train[int(list1[i]):int(list1[i+1])] = encoded_images_train[i]

# overall_encoded_train = overall_encoded_train.reshape((overall_encoded_train.shape[0], 8, 8, 1))

In [ ]:
model2 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', input_shape=(8,8,1)),
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.6),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax')
])

model2.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
print(overall_encoded_train.shape)

In [ ]:
history = model2.fit(overall_encoded_train, y_train,
                    batch_size=64,
                    epochs=10)

In [ ]:
# overall_encoded_test = np.zeros((10000,8,8,1))
# for i in range(10):
#   overall_encoded_test[list2[i]:list2[i+1]]= autoencoder_list[i].encoder(x_test[list2[i]:list2[i+1]]).numpy().reshape(x_test[list2[i]:list2[i+1]].shape[0],8,8,1)

In [ ]:
overall_encoded_test = np.zeros((10000,8,8,1))
for i in range(10):
  overall_encoded_test[list2[i]:list2[i+1]]= autoencoder_list[i].encoder(x_test[list2[i]:list2[i+1]]).numpy().reshape(x_test[list2[i]:list2[i+1]].shape[0],8,8,1)

In [ ]:
test_loss, test_acc = model2.evaluate(overall_encoded_test, y_test)